# Landing Zone Detection with RF-DETR
## CS4824 Final Project - Google Colab Training Notebook

This notebook trains an RF-DETR model to detect landing zones from Blender-rendered frames.

### Instructions:
1. Run this notebook with GPU enabled: Runtime → Change runtime type → GPU (T4)
2. The notebook will clone the GitHub repo, unzip the dataset, and train the model
3. After training, the model will be saved to Google Drive

## 1. Setup and Installation

First, check GPU availability and install dependencies.

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. Clone GitHub Repository

Clone the project repository containing the dataset and scripts.

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/PieLord757/CS4824FinalProject.git
%cd CS4824FinalProject

# List contents
!ls -la

## 3. Unzip Dataset & Install RF-DETR

Extract the COCO dataset and install the RF-DETR library.

In [ ]:
# Unzip the dataset
import zipfile
import os

zip_path = 'coco-dataset.zip'
extract_path = '.'

print("Extracting dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("✓ Dataset extracted!")

# Verify extraction
dataset_path = 'coco-dataset'
if os.path.exists(dataset_path):
    print(f"✓ Dataset directory exists: {dataset_path}")
    for split in ['train', 'valid', 'test']:
        split_path = os.path.join(dataset_path, split)
        if os.path.exists(split_path):
            num_files = len(os.listdir(split_path))
            print(f"  {split}: {num_files} files")
else:
    print("✗ Dataset extraction failed")

## 4. Install RF-DETR

Clone and install the RF-DETR library from Roboflow.

In [ ]:
# Install RF-DETR
!pip install rfdetr

print("\n✓ RF-DETR installed successfully!")

## 5. Verify Dataset Structure

Check that the COCO dataset is properly structured.

In [ ]:
import os
import json

# Check dataset structure
dataset_path = "coco-dataset"

print("Dataset structure:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")

# Check annotation files
print("\n" + "="*50)
for split in ['train', 'valid', 'test']:
    anno_file = f"{dataset_path}/{split}/_annotations.coco.json"
    if os.path.exists(anno_file):
        with open(anno_file, 'r') as f:
            data = json.load(f)
        print(f"\n{split.upper()} split:")
        print(f"  Images: {len(data['images'])}")
        print(f"  Annotations: {len(data['annotations'])}")
        print(f"  Categories: {data['categories']}")

## 6. Visualize Sample Images

Display some training images with their bounding boxes.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle

# Load training annotations
with open('coco-dataset/train/_annotations.coco.json', 'r') as f:
    train_data = json.load(f)

# Create image_id to annotations mapping
img_to_annos = {}
for anno in train_data['annotations']:
    img_id = anno['image_id']
    if img_id not in img_to_annos:
        img_to_annos[img_id] = []
    img_to_annos[img_id].append(anno)

# Display first 6 images with annotations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_info in enumerate(train_data['images'][:6]):
    img_path = f"coco-dataset/train/{img_info['file_name']}"
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    ax = axes[idx]
    ax.imshow(img)
    
    # Draw bounding boxes
    if img_info['id'] in img_to_annos:
        for anno in img_to_annos[img_info['id']]:
            x, y, w, h = anno['bbox']
            rect = Rectangle((x, y), w, h, linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
    
    ax.set_title(f"{img_info['file_name']}")
    ax.axis('off')

plt.tight_layout()
plt.show()

## 7. Train the RF-DETR Model

Train the model using the RF-DETR library.

In [ ]:
import torch
from rfdetr import RFDETRNano

def train_landing_zone_model():
    """Train RF-DETR model for landing zone detection"""
    
    dataset_path = "coco-dataset"
    
    # Initialize model
    print("Initializing RF-DETR Nano model...")
    model = RFDETRNano()
    
    print(f"\nTraining Configuration:")
    print(f"  Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
    print(f"  Dataset: {dataset_path}")
    
    # Train model
    print("\nStarting training...")
    model.train(
        dataset_dir=dataset_path,
        epochs=100,
        batch_size=8,
        grad_accum_steps=4,
        lr=1e-4
    )
    
    print("\n✓ Training completed!")
    
    return model

# Run training
model = train_landing_zone_model()

## 8. Check Training Outputs

List the model checkpoints saved during training.

In [ ]:
# Check for training outputs
import os
import glob

# Find .pth files (model checkpoints)
pth_files = glob.glob("**/*.pth", recursive=True)
print("Model checkpoints found:")
for f in pth_files:
    size = os.path.getsize(f) / (1024*1024)
    print(f"  {f} ({size:.2f} MB)")

## 9. Test Inference

Run inference on test images to see model predictions.

In [ ]:
# Test inference on sample images
from PIL import Image
import matplotlib.pyplot as plt

# Get test images
test_dir = "coco-dataset/test"
test_images = [f for f in os.listdir(test_dir) if f.endswith('.png')][:6]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_name in enumerate(test_images):
    img_path = os.path.join(test_dir, img_name)
    
    # Run inference
    detections = model.predict(img_path, threshold=0.3)
    
    # Load and display image
    img = Image.open(img_path)
    ax = axes[idx]
    ax.imshow(img)
    
    # Draw detections
    for det in detections:
        x1, y1, x2, y2 = det.xyxy
        conf = det.confidence
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                             linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        ax.text(x1, y1-5, f"{conf:.2f}", color='red', fontsize=10, weight='bold')
    
    ax.set_title(img_name)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 10. Save Model to Google Drive

Mount Google Drive and save the trained model for later use.

In [ ]:
# Mount Google Drive to save the model
from google.colab import drive
drive.mount('/content/drive')

# Save model to Drive
import shutil
import glob

drive_output = "/content/drive/MyDrive/landing_zone_model"
os.makedirs(drive_output, exist_ok=True)

# Find and copy model files
pth_files = glob.glob("**/*.pth", recursive=True)
for pth_file in pth_files:
    dst = os.path.join(drive_output, os.path.basename(pth_file))
    shutil.copy2(pth_file, dst)
    print(f"✓ Saved: {os.path.basename(pth_file)}")

print(f"\n✓ Model saved to Google Drive: {drive_output}")